In [1]:
import numpy as np
import sys
from PIL import Image, ImageEnhance
import matplotlib.patches as patches
import matplotlib.pyplot as plt 
import time

In [ ]:
from darknet import *

In [ ]:

datacfg = 'cfg/sdss.data'
cfgfile = 'cfg/sdss.cfg'

weightfile = 'sdss.weights' 
filename = 'galaxies.jpg' ##try with galaxies2.jpg and galaxies3.jpg
                          ##The three images were downloaded from google images

thresh = 0.1  #detection probability threshold
hier_thresh = 0.5

net = load_net(cfgfile, weightfile, 0)
meta = load_meta(datacfg)
outputsA = detect(net, meta, filename,thresh=thresh)
print outputsA

In [ ]:
centers = []
outputs = []
for output in outputsA:
    check= True
    center_output = np.array([output[2][0],output[2][1],output[2][2],output[2][3]])
##Yolo3 tends to detect galaxies more than once and in different sized detection boxes

    for center in centers:
        if np.linalg.norm(center_output - center)<15:
            check = False
    if check:
        centers.append(center_output)
        outputs.append(output)

        
print '%d galaxies detected'%len(outputs)
    
asd=len(outputs)

In [ ]:

#plot image and detections
img = Image.open(filename)
contrast = ImageEnhance.Contrast(img)
img2 = contrast.enhance(1.5) # just for visualzation 
fig,ax = plt.subplots(figsize=(30,22))
plt.axis('off')
plt.tight_layout(pad=0)
plt.imshow(img2)
ax.set_aspect('equal')
for output in outputs:
    
        cx=output[2][0]
        cy=output[2][1]
        w=output[2][2]
        h=output[2][3]
        
        r=cx+w/2
        l=cx-w/2
        t=cy-h/2
        b=cy+h/2
        rect = patches.Rectangle((l-4,t-3),r-l+8,b-t+4,linewidth=1,edgecolor='b',facecolor='none')      
        ax.add_patch(rect)
        ax.annotate(output[0],((l+r)/2-30,t-5),color='w',fontsize=14)
plt.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=0, hspace=0) 
#plt.savefig('../writable/%s_thresh_%4.2f_weights_%s_detections_%4d.jpg'%(filename[3:-4],thresh,weightfile[14:-8],asd),dpi=180)
plt.savefig('../galaxias_yolo3.jpg',dpi=180)
plt.show()